In [1]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# The raw code for this IPython notebook is by default hidden for easier reading.
# To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')




In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
import random
import json
from math import ceil
%matplotlib inline
import os
import matplotlib as mpl
from PIL import Image
from io import BytesIO
from bayescmd.abc import SummaryStats
from bayescmd.abc import import_actual_data
from bayescmd.abc import inputParse
import scipy.stats as stats
import statsmodels.api as sm


import os
from pathlib import Path
from distutils import dir_util
from pprint import pprint
import pickle

# BayesCMD packages 

from bayescmd.abc import import_actual_data
from bayescmd.abc import priors_creator

# Google BigQuery
from google.cloud import bigquery
%load_ext google.cloud.bigquery


from bayescmd.results_handling import get_output

mpl.rc('figure', dpi=300, figsize=(7.5,8))

mpl.rcParams["xtick.labelsize"]= 8

mpl.rcParams["ytick.labelsize"]= 8
    
mpl.rcParams["axes.labelsize"]= 10

mpl.rcParams["figure.titlesize"] = 12

STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD


In [3]:
def TIFF_exporter(fig, fname, fig_dir='.', extra_artists=()):
    """
    Parameters
    ----------
    fig: matplotlib figure
    """

    # save figure
    # (1) save the image in memory in PNG format
    png1 = BytesIO()
    fig.savefig(png1, format='png', bbox_inches='tight', bbox_extra_artists=extra_artists,
                dpi=300, transparent=False)

    # (2) load this image into PIL
    png2 = Image.open(png1)

    # (3) save as TIFF
    png2.save(os.path.join(fig_dir, '{}.tiff'.format(fname)),
              compression='tiff_deflate')
    png1.close()
    return True

In [4]:
# Explicitly use service account credentials by specifying the private
# key file. All clients in google-cloud-python have this helper.
client = bigquery.Client.from_service_account_json(
    "../../gcloud/hypothermia-auth.json"
)

In [5]:
def generate_posterior_query(project, dataset, model, distance, parameters, limit=50000):
    unpacked_params = ",\n".join(parameters)
    histogram_query = """
SELECT
    {unpacked_params},
    {distance},
    idx
FROM
  `{project}.{dataset}.{model}`
ORDER BY
  {distance} ASC
LIMIT
  {limit}
    """.format(project=project, dataset=dataset, model=model, unpacked_params=unpacked_params,distance=distance, limit=limit)
    return histogram_query

In [6]:
def load_configuration(model_version, dataset, verbose=False):
    current_file = Path(os.path.abspath(''))
    config_file = os.path.join(current_file.parents[2],
                              'config_files',
                               'abc',
                               'bp_hypothermia_{}'.format(model_version),
                               'bp_hypothermia_{}_config.json'.format(model_version)
                              )

    with open(config_file, 'r') as conf_f:
        conf = json.load(conf_f)

    params = conf['priors']

    input_path = os.path.join(current_file.parents[2],
                              'data',
                              'clean_hypothermia',
                              '{}_filtered_formatted.csv'.format(dataset.upper()))

    d0 = import_actual_data(input_path)

    targets = conf['targets']
    model_name = conf['model_name']
    inputs = conf['inputs']

    config = {
        "model_name": model_name,
        "targets": targets,
        "times": d0['t'],
        "inputs": inputs,
        "parameters": params,
        "input_path": input_path,
        "zero_flag": conf['zero_flag'],
    }
    
    if verbose:
        pprint(config)
        
    return config, d0

# Read in the posterior #


In [7]:
labels = {"t": "Time (sec)", 
 "HbO2": "$\Delta$HbO2 $(\mu M)$",
 "HHb": "$\Delta$HHb $(\mu M)$",
 "CCO": "$\Delta$CCO $(\mu M)$"}

signals=['HbO2', 'HHb','CCO']
ticker_step = [20, 10, 10, 10, 0.5]
colpal = sns.color_palette(n_colors=len(signals))

In [8]:
def get_runs(posterior, conf, n_repeats=50): 
    rand_selection = random.sample(range(posterior.shape[0]), n_repeats)
    outputs_list = []
    p_names = list(conf['parameters'].keys())
    posteriors = posterior[p_names].values
    d0 = import_actual_data(conf['input_path'])
    input_data = inputParse(d0, conf['inputs'])
    while len(outputs_list) < n_repeats:
        idx = rand_selection.pop()
        print("\tSample {}, idx:{}".format(len(outputs_list), idx))
        p = dict(zip(p_names, posteriors[idx]))
            
    
        _, output = get_output(
            conf['model_name'],
            p,
            conf['times'],
            input_data,
            d0,
            conf['targets'],
            distance="NRMSE",
            zero_flag=conf['zero_flag'])
        outputs_list.append(output)
    return outputs_list


## Generating posterior predictive ##

We can sample directly from the posterior to generate our posterior predictive.We then generate a variety of potentially useful summary statistics as well as the residuals, autocorrelation of the signals and autocorrelation of the residuals for each signal.

We also generate each summary statistic for the observed data so as to compare this with the posterior predictive distribution of these statistics.

In [9]:
configuration = {}
model_data_combos = {"LWP475": ["1","2","4"],
                    "LWP479": ["1_1", "2_1", "4_1"]}
for combo in [(m,d) for d, l in model_data_combos.items() for m in l]:
    print("Working on (bph{}, {})".format(*combo))
    model_number = combo[0]
    model_name = 'bph{}'.format(model_number)
    DATASET = combo[1]
    configuration[model_name] = {}
    
    configuration[model_name][DATASET] = {}
    config, d0 = load_configuration(model_number, DATASET)
    configuration[model_name][DATASET]['bayescmd_config'] = config
    configuration[model_name][DATASET]['original_data']= d0

    configuration[model_name][DATASET]['posterior_query'] = generate_posterior_query('hypothermia-bayescmd', 
                                                                                     DATASET, 
                                                                                     model_name, 
                                                                                     'NRMSE', 
                                                                                     list(configuration[model_name][DATASET]['bayescmd_config']['parameters'].keys()),
                                                                                     limit=5000)
    config = configuration[model_name][DATASET]['bayescmd_config']
    figPath = "/home/buck06191/Dropbox/phd/hypothermia/ABC/Figures/{}/{}/{}".format(model_name, DATASET, 'NRMSE')
    dir_util.mkpath(figPath)

    # Get posterior
    print("\tRunning SQL query")
    df_post = client.query(configuration[model_name][DATASET]['posterior_query']).to_dataframe()
    N=5000
    print("\tSampling from the posterior {} times.".format(N))
    
    outputs_list = get_runs(df_post, config, n_repeats=N)
    results = {}
    print("\n")
    
    for i, output in enumerate(outputs_list):
        results[i] = {}
        summary_creator = SummaryStats(output, config['targets'], config['zero_flag'], observed_data = d0)
        summary_creator.get_stats()
        results[i]['data'] = summary_creator.d0
        results[i]['residuals'] = summary_creator.residuals
        results[i]['stats'] = summary_creator.summary_stats

    resid_formatted = [{'Batch': i, 'Signal': j, 'Residuals': v, 'Time (sec)': idx+1} for i in results.keys(
    ) for j in results[i]['residuals'].keys() for idx, v in enumerate(results[i]['residuals'][j])]
    residuals = pd.DataFrame(resid_formatted)
    fig1, axes1 = plt.subplots(2,2, figsize=(7,7))
    fig2, axes2 = plt.subplots(2,2, figsize=(7,7))

    for ii, s in enumerate(config['targets']):
        signal_data=residuals[residuals['Signal']==s]['Residuals']
        ax1=axes1.flatten()[ii]
        sns.distplot(signal_data, ax=ax1)
        resid_mu, resid_sigma = np.mean(signal_data), np.std(signal_data)
        print("\t{}: Mean $(\mu$): {:.3g}\n\tStandard Deviation ($\sigma$): {:.3g}".format(s.upper(),resid_mu, resid_sigma))
        mean = ax1.axvline(resid_mu, color='k', label='Mean', linestyle='--')
        std = ax1.axvline(resid_mu-resid_sigma, color='g', label='Standard Deviation', linestyle='--')
        ax1.axvline(resid_mu+resid_sigma, color='g', linestyle='--')
        ax1.set_title("{}".format(s), fontsize=12)


        ax2=axes2.flatten()[ii]
        resid = signal_data.values
        sm.qqplot(resid, line='s',ax=ax2)
        ax2.axhline(0, color='k', linestyle='--')
        sample_mean = ax2.axhline(resid_mu, color='xkcd:orange', linestyle=':', label="Sample Mean")
        theoretical_mean = ax2.axvline(0, color='k', linestyle='--', label="Theoretical Mean")
        ax2.set_title("{}".format(s), fontsize=12)
        # print(stats.anderson(resid,dist='norm'))
    axes1[-1, -1].axis('off')
    axes2[-1, -1].axis('off')
    lgd1 = fig1.legend(handles = [mean, std], bbox_to_anchor=(0.55, 0.4), loc=2, fontsize=14)
    fig1.tight_layout()
    fig1.subplots_adjust(top=0.85)
    TIFF_exporter(fig1, 'residuals_dist', fig_dir=figPath, extra_artists=(lgd1,))

    lgd2 = fig2.legend(handles = [theoretical_mean, sample_mean], bbox_to_anchor=(0.55, 0.4), loc=2, fontsize=14)
    fig2.tight_layout()
    fig2.subplots_adjust(top=0.85)
    TIFF_exporter(fig2, 'residuals_qq', fig_dir=figPath, extra_artists=(lgd2,))

    posterior = {}
    prior = {}
    entropy = {}
    bins = {}
    fig3, axes3 = plt.subplots(ceil(len(config['parameters'])/3),3, figsize=(7,8))
    i = 0
    for k,v in config['parameters'].items():
        ax = axes3[i//3][i%3]

        prior[k], bins[k] = np.histogram(np.random.uniform(v[1][0], v[1][1], 5000), 50, density=True)
        posterior[k], _ = np.histogram(df_post[k].values, bins=bins[k], density=True)

        entropy[k] = stats.entropy(posterior[k], prior[k])
        line_post = ax.bar(bins[k][:-1], posterior[k], width = bins[k][1]-bins[k][0], align='edge', label='Posterior')
        line_prior = ax.bar(bins[k][:-1], prior[k], width = bins[k][1]-bins[k][0], align='edge', alpha=.75, label='Prior')
        #ax.text(0.7,0.965, "Entropy: {:.3g}".format(entropy[k]), transform=ax.transAxes, size=16)
        ax.set_title("K-L Divergence: {:.3g}".format(entropy[k]), y=1.01, fontsize=12)
        ax.set_xlabel(k)

        fig3.tight_layout()
        i+=1

    n_emptyAxes = 3-len(config['parameters'])%3
    if n_emptyAxes > 0:
        for n in range(1, n_emptyAxes+1):
            axes3[-1, int(-1*n)].axis('off')

    # axes[-1, -2].axis('off')
    # axes3[-1, -1].axis('off')

    lgd3 = fig3.legend(handles=[line_post, line_prior], bbox_to_anchor=(0.7, 0.2), loc=2, fontsize=12)

    TIFF_exporter(fig3, 'kl_div', fig_dir=figPath, extra_artists=(lgd3,))
    plt.close('all')

Working on (bph1, LWP475)
	Running SQL query
	Sampling from the posterior 5000 times.
	Sample 0, idx:1809
	Sample 1, idx:4432
	Sample 2, idx:537
	Sample 3, idx:3650
	Sample 4, idx:225
	Sample 5, idx:1950
	Sample 6, idx:2029
	Sample 7, idx:2933
	Sample 8, idx:2595
	Sample 9, idx:4783
	Sample 10, idx:3488
	Sample 11, idx:3584
	Sample 12, idx:3563
	Sample 13, idx:366
	Sample 14, idx:1216
	Sample 15, idx:1885
	Sample 16, idx:676
	Sample 17, idx:2544
	Sample 18, idx:342
	Sample 19, idx:2708
	Sample 20, idx:1388
	Sample 21, idx:2543
	Sample 22, idx:4300
	Sample 23, idx:4972
	Sample 24, idx:4422
	Sample 25, idx:1091
	Sample 26, idx:2462
	Sample 27, idx:3118
	Sample 28, idx:3417
	Sample 29, idx:311
	Sample 30, idx:1762
	Sample 31, idx:1164
	Sample 32, idx:3829
	Sample 33, idx:4796
	Sample 34, idx:1321
	Sample 35, idx:3281
	Sample 36, idx:1969
	Sample 37, idx:4255
	Sample 38, idx:1424
	Sample 39, idx:1547
	Sample 40, idx:3599
	Sample 41, idx:4917
	Sample 42, idx:1180
	Sample 43, idx:4058
	Sampl

	Sample 377, idx:839
	Sample 378, idx:2567
	Sample 379, idx:158
	Sample 380, idx:4372
	Sample 381, idx:4402
	Sample 382, idx:1879
	Sample 383, idx:1385
	Sample 384, idx:2174
	Sample 385, idx:2230
	Sample 386, idx:4663
	Sample 387, idx:3236
	Sample 388, idx:2374
	Sample 389, idx:1936
	Sample 390, idx:4653
	Sample 391, idx:897
	Sample 392, idx:2123
	Sample 393, idx:2743
	Sample 394, idx:4893
	Sample 395, idx:2383
	Sample 396, idx:607
	Sample 397, idx:3942
	Sample 398, idx:1377
	Sample 399, idx:3868
	Sample 400, idx:2432
	Sample 401, idx:830
	Sample 402, idx:2402
	Sample 403, idx:1718
	Sample 404, idx:2950
	Sample 405, idx:2112
	Sample 406, idx:3981
	Sample 407, idx:3009
	Sample 408, idx:2070
	Sample 409, idx:3659
	Sample 410, idx:2089
	Sample 411, idx:1941
	Sample 412, idx:1672
	Sample 413, idx:266
	Sample 414, idx:832
	Sample 415, idx:694
	Sample 416, idx:775
	Sample 417, idx:4390
	Sample 418, idx:3339
	Sample 419, idx:4506
	Sample 420, idx:3037
	Sample 421, idx:422
	Sample 422, idx:685

	Sample 754, idx:2713
	Sample 755, idx:526
	Sample 756, idx:590
	Sample 757, idx:3053
	Sample 758, idx:3550
	Sample 759, idx:2430
	Sample 760, idx:993
	Sample 761, idx:1062
	Sample 762, idx:538
	Sample 763, idx:3993
	Sample 764, idx:1878
	Sample 765, idx:3516
	Sample 766, idx:2308
	Sample 767, idx:2312
	Sample 768, idx:3195
	Sample 769, idx:552
	Sample 770, idx:1541
	Sample 771, idx:3277
	Sample 772, idx:632
	Sample 773, idx:1428
	Sample 774, idx:1294
	Sample 775, idx:1540
	Sample 776, idx:4381
	Sample 777, idx:4349
	Sample 778, idx:4801
	Sample 779, idx:1864
	Sample 780, idx:2782
	Sample 781, idx:1818
	Sample 782, idx:79
	Sample 783, idx:3365
	Sample 784, idx:3794
	Sample 785, idx:1065
	Sample 786, idx:344
	Sample 787, idx:0
	Sample 788, idx:3039
	Sample 789, idx:1949
	Sample 790, idx:2178
	Sample 791, idx:3226
	Sample 792, idx:1116
	Sample 793, idx:2251
	Sample 794, idx:3329
	Sample 795, idx:2666
	Sample 796, idx:2877
	Sample 797, idx:4590
	Sample 798, idx:4620
	Sample 799, idx:2119


	Sample 1125, idx:238
	Sample 1126, idx:723
	Sample 1127, idx:3246
	Sample 1128, idx:4516
	Sample 1129, idx:1787
	Sample 1130, idx:556
	Sample 1131, idx:2611
	Sample 1132, idx:1244
	Sample 1133, idx:1172
	Sample 1134, idx:3798
	Sample 1135, idx:1021
	Sample 1136, idx:1027
	Sample 1137, idx:2399
	Sample 1138, idx:2345
	Sample 1139, idx:305
	Sample 1140, idx:1300
	Sample 1141, idx:3212
	Sample 1142, idx:4587
	Sample 1143, idx:923
	Sample 1144, idx:3863
	Sample 1145, idx:499
	Sample 1146, idx:2835
	Sample 1147, idx:2212
	Sample 1148, idx:4771
	Sample 1149, idx:1745
	Sample 1150, idx:3159
	Sample 1151, idx:2391
	Sample 1152, idx:2175
	Sample 1153, idx:1235
	Sample 1154, idx:2591
	Sample 1155, idx:2205
	Sample 1156, idx:330
	Sample 1157, idx:2639
	Sample 1158, idx:1195
	Sample 1159, idx:1150
	Sample 1160, idx:3275
	Sample 1161, idx:926
	Sample 1162, idx:1251
	Sample 1163, idx:299
	Sample 1164, idx:2890
	Sample 1165, idx:3136
	Sample 1166, idx:4842
	Sample 1167, idx:3232
	Sample 1168, idx:29

	Sample 1486, idx:725
	Sample 1487, idx:2326
	Sample 1488, idx:1117
	Sample 1489, idx:4729
	Sample 1490, idx:3301
	Sample 1491, idx:2338
	Sample 1492, idx:3764
	Sample 1493, idx:1769
	Sample 1494, idx:1429
	Sample 1495, idx:1715
	Sample 1496, idx:3581
	Sample 1497, idx:790
	Sample 1498, idx:4110
	Sample 1499, idx:3631
	Sample 1500, idx:4526
	Sample 1501, idx:3691
	Sample 1502, idx:1774
	Sample 1503, idx:2438
	Sample 1504, idx:12
	Sample 1505, idx:3447
	Sample 1506, idx:144
	Sample 1507, idx:4344
	Sample 1508, idx:4817
	Sample 1509, idx:1622
	Sample 1510, idx:1074
	Sample 1511, idx:1492
	Sample 1512, idx:4369
	Sample 1513, idx:2944
	Sample 1514, idx:753
	Sample 1515, idx:3815
	Sample 1516, idx:2831
	Sample 1517, idx:4865
	Sample 1518, idx:1616
	Sample 1519, idx:2005
	Sample 1520, idx:4559
	Sample 1521, idx:4864
	Sample 1522, idx:3083
	Sample 1523, idx:2721
	Sample 1524, idx:2069
	Sample 1525, idx:4441
	Sample 1526, idx:3781
	Sample 1527, idx:4680
	Sample 1528, idx:4608
	Sample 1529, idx

...

	Sample 3319, idx:3331
	Sample 3320, idx:3916
	Sample 3321, idx:3861
	Sample 3322, idx:3948
	Sample 3323, idx:3080
	Sample 3324, idx:143
	Sample 3325, idx:1420
	Sample 3326, idx:3891
	Sample 3327, idx:4873
	Sample 3328, idx:1765
	Sample 3329, idx:2817
	Sample 3330, idx:3914
	Sample 3331, idx:1135
	Sample 3332, idx:1861
	Sample 3333, idx:3125
	Sample 3334, idx:1994
	Sample 3335, idx:3140
	Sample 3336, idx:1114
	Sample 3337, idx:4579
	Sample 3338, idx:4412
	Sample 3339, idx:1030
	Sample 3340, idx:256
	Sample 3341, idx:474
	Sample 3342, idx:3732
	Sample 3343, idx:1846
	Sample 3344, idx:3133
	Sample 3345, idx:3084
	Sample 3346, idx:2626
	Sample 3347, idx:1391
	Sample 3348, idx:4901
	Sample 3349, idx:4486
	Sample 3350, idx:4660
	Sample 3351, idx:1943
	Sample 3352, idx:3428
	Sample 3353, idx:1724
	Sample 3354, idx:537
	Sample 3355, idx:1695
	Sample 3356, idx:2682
	Sample 3357, idx:4079
	Sample 3358, idx:3165
	Sample 3359, idx:1423
	Sample 3360, idx:1974
	Sample 3361, idx:1743
	Sample 3362, i

	Sample 3679, idx:1358
	Sample 3680, idx:3832
	Sample 3681, idx:2793
	Sample 3682, idx:1372
	Sample 3683, idx:202
	Sample 3684, idx:679
	Sample 3685, idx:277
	Sample 3686, idx:2082
	Sample 3687, idx:1149
	Sample 3688, idx:1630
	Sample 3689, idx:2117
	Sample 3690, idx:3686
	Sample 3691, idx:4918
	Sample 3692, idx:938
	Sample 3693, idx:3378
	Sample 3694, idx:2227
	Sample 3695, idx:4947
	Sample 3696, idx:3058
	Sample 3697, idx:4181
	Sample 3698, idx:3649
	Sample 3699, idx:3134
	Sample 3700, idx:1534
	Sample 3701, idx:4382
	Sample 3702, idx:1862
	Sample 3703, idx:857
	Sample 3704, idx:3375
	Sample 3705, idx:1828
	Sample 3706, idx:4668
	Sample 3707, idx:754
	Sample 3708, idx:3560
	Sample 3709, idx:1474
	Sample 3710, idx:696
	Sample 3711, idx:2947
	Sample 3712, idx:2498
	Sample 3713, idx:3765
	Sample 3714, idx:3102
	Sample 3715, idx:963
	Sample 3716, idx:4955
	Sample 3717, idx:2156
	Sample 3718, idx:3978
	Sample 3719, idx:4184
	Sample 3720, idx:2028
	Sample 3721, idx:3880
	Sample 3722, idx:6

	Sample 4039, idx:2746
	Sample 4040, idx:4346
	Sample 4041, idx:616
	Sample 4042, idx:1066
	Sample 4043, idx:121
	Sample 4044, idx:3210
	Sample 4045, idx:774
	Sample 4046, idx:3218
	Sample 4047, idx:2821
	Sample 4048, idx:1721
	Sample 4049, idx:1459
	Sample 4050, idx:2612
	Sample 4051, idx:3563
	Sample 4052, idx:2116
	Sample 4053, idx:2335
	Sample 4054, idx:3287
	Sample 4055, idx:2342
	Sample 4056, idx:1297
	Sample 4057, idx:717
	Sample 4058, idx:1813
	Sample 4059, idx:871
	Sample 4060, idx:2702
	Sample 4061, idx:4299
	Sample 4062, idx:1555
	Sample 4063, idx:1905
	Sample 4064, idx:1750
	Sample 4065, idx:2129
	Sample 4066, idx:1529
	Sample 4067, idx:3320
	Sample 4068, idx:2245
	Sample 4069, idx:2167
	Sample 4070, idx:1468
	Sample 4071, idx:12
	Sample 4072, idx:4572
	Sample 4073, idx:1664
	Sample 4074, idx:2696
	Sample 4075, idx:732
	Sample 4076, idx:2611
	Sample 4077, idx:4613
	Sample 4078, idx:3599
	Sample 4079, idx:4816
	Sample 4080, idx:779
	Sample 4081, idx:4762
	Sample 4082, idx:47

	Sample 4399, idx:2358
	Sample 4400, idx:3679
	Sample 4401, idx:75
	Sample 4402, idx:3098
	Sample 4403, idx:4775
	Sample 4404, idx:4953
	Sample 4405, idx:4682
	Sample 4406, idx:3954
	Sample 4407, idx:2327
	Sample 4408, idx:3893
	Sample 4409, idx:4116
	Sample 4410, idx:3569
	Sample 4411, idx:3024
	Sample 4412, idx:2753
	Sample 4413, idx:4052
	Sample 4414, idx:3598
	Sample 4415, idx:4243
	Sample 4416, idx:1803
	Sample 4417, idx:436
	Sample 4418, idx:2012
	Sample 4419, idx:3203
	Sample 4420, idx:3382
	Sample 4421, idx:3627
	Sample 4422, idx:884
	Sample 4423, idx:1983
	Sample 4424, idx:1830
	Sample 4425, idx:3408
	Sample 4426, idx:1973
	Sample 4427, idx:960
	Sample 4428, idx:1704
	Sample 4429, idx:3547
	Sample 4430, idx:4676
	Sample 4431, idx:3234
	Sample 4432, idx:4619
	Sample 4433, idx:1686
	Sample 4434, idx:977
	Sample 4435, idx:168
	Sample 4436, idx:1076
	Sample 4437, idx:1121
	Sample 4438, idx:3256
	Sample 4439, idx:2511
	Sample 4440, idx:3851
	Sample 4441, idx:3048
	Sample 4442, idx:

	Sample 4759, idx:1719
	Sample 4760, idx:1667
	Sample 4761, idx:2894
	Sample 4762, idx:2187
	Sample 4763, idx:251
	Sample 4764, idx:1644
	Sample 4765, idx:2325
	Sample 4766, idx:650
	Sample 4767, idx:598
	Sample 4768, idx:922
	Sample 4769, idx:3894
	Sample 4770, idx:1144
	Sample 4771, idx:4936
	Sample 4772, idx:1189
	Sample 4773, idx:1867
	Sample 4774, idx:3442
	Sample 4775, idx:2509
	Sample 4776, idx:3070
	Sample 4777, idx:3183
	Sample 4778, idx:2959
	Sample 4779, idx:2112
	Sample 4780, idx:4339
	Sample 4781, idx:1235
	Sample 4782, idx:2854
	Sample 4783, idx:4631
	Sample 4784, idx:1935
	Sample 4785, idx:4322
	Sample 4786, idx:102
	Sample 4787, idx:4745
	Sample 4788, idx:1061
	Sample 4789, idx:2697
	Sample 4790, idx:2331
	Sample 4791, idx:2164
	Sample 4792, idx:3827
	Sample 4793, idx:1975
	Sample 4794, idx:3448
	Sample 4795, idx:356
	Sample 4796, idx:4358
	Sample 4797, idx:3139
	Sample 4798, idx:1308
	Sample 4799, idx:4411
	Sample 4800, idx:1713
	Sample 4801, idx:847
	Sample 4802, idx:

# Residuals #
We can also look at the residuals directly. We can see from both the time series and the distributions of the residuals that they are generally normally distributed aorund 0, with the exception of HbT which is centred roughly on -1.5, showing the general under estimation of the signal. 

In [10]:
# g = sns.FacetGrid(data=residuals, row = 'Signal', height = 4, aspect = 3, sharey=False,sharex=False)



## Entropy of prior to posterior ##

We need to sample $N_{limit}$ times from the prior distributions for each parameter. $N_{limit}$ is the number of samples in our posterior, in this case that is 3000. 

We then need to bin our posteriors and priors and the calculate the divergence.

In [11]:
n_emptyAxes = 3-len(config['parameters'])%3
if n_emptyAxes > 0:
    for n in range(1, n_emptyAxes+1):
        print(n)
        #axes[-1, int(-1*n)].axis('off')

1
2


In [12]:
x = (1,2)
print("{},{}".format(*x))

1,2
